<a href="https://colab.research.google.com/github/A-NGJ/AudioExplorers2023/blob/main/%20AudioExplorers2023_Transformer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Dropout, LayerNormalization, Embedding, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
import numpy as np
import cv2

import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Dropout, LayerNormalization, Embedding
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
import numpy as np
import cv2
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score

from keras.preprocessing.text import Tokenizer
from tensorflow.keras.optimizers.schedules import ExponentialDecay



def resize_data(data):
    resized_data = np.zeros((data.shape[0], 32, 96))
    for i, img in enumerate(data):
        resized_img = cv2.resize(img, (96, 32))
        resized_data[i] = resized_img
    return resized_data


# Load data
X = np.load('training.npy')
y = np.load('training_labels.npy')

# Preprocess input data
X = resize_data(X)


# Define model parameters
max_length = X.shape[1]
input_dim = X.shape[2]
output_dim = len(np.unique(y))
d_model = 96
num_heads = 2
dff = 128
num_layers = 2
dropout_rate = 0.2

# Train-validation split
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=1)


# One-hot encode labels
num_classes = 5
y_train = to_categorical(y_train, num_classes=num_classes)
y_val = to_categorical(y_val, num_classes=num_classes)


# Define Transformer encoder layer
class TransformerEncoder(tf.keras.layers.Layer):
    def __init__(self, d_model, num_heads, dff, rate=0.1):
        super().__init__()

        self.mha = tf.keras.layers.MultiHeadAttention(num_heads=num_heads, key_dim=d_model)
        self.dropout1 = tf.keras.layers.Dropout(rate)
        self.layernorm1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)

        self.ffn = tf.keras.Sequential([
            tf.keras.layers.Dense(dff, activation='relu'),
            tf.keras.layers.Dense(d_model)
        ])
        # print('Last Dense layer units:')
        # print(self.ffn.layers[-1].units)
        # print('d_model:')
        # print(d_model)
        self.dropout2 = tf.keras.layers.Dropout(rate)
        self.layernorm2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)



    def call(self, x, training):
        attn_output = self.mha(x, x, x)  # Multi-head attention
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(x + attn_output)  # Add and normalize

        ffn_output = self.ffn(out1)  # Feed-forward neural network
        ffn_output = self.dropout2(ffn_output, training=training)
        out2 = self.layernorm2(out1 + ffn_output)  # Add and normalize

        return out2

# Define input layer
inputs = Input(shape=(max_length, input_dim))

# Add Transformer encoder layers
x = inputs
for i in range(num_layers):
    transformer_encoder = TransformerEncoder(d_model, num_heads, dff, dropout_rate)
    x = transformer_encoder(x, training=True)

# Flatten and add output layer
x = Flatten()(x)
outputs = Dense(output_dim, activation='softmax')(x)

# Define and compile model
model = Model(inputs=inputs, outputs=outputs)
lr_schedule = ExponentialDecay(initial_learning_rate=1e-3, decay_steps=10000, decay_rate=0.9)
optimizer = Adam(learning_rate=lr_schedule)
#optimizer = Adam(lr=0.001)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])


# Train the model
model.fit(X_train, y_train, batch_size=32, epochs=10, validation_data=(X_val, y_val), callbacks=[EarlyStopping(patience=4)])

# model.fit(X_train, y_train, batch_size=32, epochs=10, validation_data=(X_val, y_val), callbacks=[EarlyStopping(patience=2)])

# Save the model
model.save('music_detector_transformer3.h5')